# Load a Robot into the Genesis Scene

In this tutorial, we introduce how to set up a basic Genesis environment, create a simple scene, and load a robot. We will go step by step through initialization, scene construction, simulator options, and visualizer settings.


## What You Will Learn

1. How to Initialize Genesis

* Understand the basic startup procedure using `gs.init()`.
* Select `vulkan` backend recommended for AMD GPUs.

2. How to Build a Scene

* Every simulation in Genesis occurs within a `Scene`, which integrates a **Simulator** (physics engine) and a **Visualizer** (rendering engine).
* Show the difference between GUI (`show_viewer=True`) and headless (`False`) modes.

3. How to Configure the Visualizer

* Learn to control virtual camera behavior using `ViewerOptions` (position, look-at point, and field of view).
* Adjust viewer resolution, frame rate, and threading for optimized rendering performance.

4. How to Add Entities and Robots

* Understand that all physical elements are represented as `Entity` objects, managed via `scene.add_entity()`.
* See a full example importing a **Franka Emika Panda** robotic arm and a plane into the scene, preparing for motion control.

In [1]:
# Suppress warning messages for clearer output
import warnings
import os

os.environ["TI_LOG_LEVEL"] = "error"
warnings.filterwarnings("ignore")

## Backend Initialization

Genesis supports multiple backends for parallel simulation. For AMD hardware, the recommended backend is Vulkan, a low-level API for high performance GPU programming.

In addition to choosing the backend, you can configure various parameters during initialization:

Example:

```python
gs.init(
    precision           = '32',
    logging_level       = None,
    backend             = gs.vulkan,
    theme               = 'dark',
)
```

Here, we use the default init settings and set backend to vulkan.

In [2]:
import genesis as gs
import numpy as np

gs.init(backend=gs.vulkan, theme='light')

[Genesis] [09:55:47] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [09:55:47] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [09:55:47] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [09:55:47] [INFO] Consider setting 'performance_mode=True' in production to maximise runtime speed, if significantly increasing compilation time is not a concern.
[Genesis] [09:55:47] [INFO] Running on [AMD Radeon Graphics] with backend gs.vulkan. Device memory: 60.75 GB.
[Genesis] [09:55:47] [INFO] 🚀 Genesis initialized. 🔖 version: 0.3.3, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'light'.


## Create a Scene

In Genesis, all entities—such as robots, objects, cameras, and sensors—exist within a **Scene**. A Scene serves as the core container of the simulation, encapsulating two main components:

* **Simulator**: Defines the physical world and handles all physics computations.
* **Visualizer**: Renders the scene and manages the graphical display.

A scene can be created either **with** or **without** a viewer:

* `show_viewer=True`: Launches a GUI viewer, useful for debugging or interactive visualization.
* `show_viewer=False`: Runs in headless mode, ideal for large-scale training or server-side execution.

In this lab, we set `show_viewer` to `False` and use the default `SimOptions` and `ViewerOptions` settings.
Later, instead of rendering the scene directly in this notebook, we will save the rendering output as a video for visualization.

Note that you can only create the scene ONCE. Recreating it will cause an ERROR. 


In [3]:
scene = gs.Scene(show_viewer=False)

[Genesis] [09:55:52] [INFO] Scene <17b8005> created.


## Load a Robot

Once a scene has been created in Genesis, the next step is to populate it with robots, objects, or other physical entities. Genesis follows a fully **object-oriented design**, where every element in the simulation world is represented as an [**`Entity`**](https://genesis-world.readthedocs.io/en/latest/api_reference/entity/index.html).

An `Entity` is the abstraction for everything in the scene that requires physics simulation. This includes rigid or deformable bodies, terrains, fluids, and, of course, robots. Entities are introduced into a scene via the function `scene.add_entity`.

The first argument to `add_entity` is a **morph**, which encapsulates both the **geometry** and **pose** of an entity.

Different morph types allow you to load entities from:

* **Shape primitives** (e.g., planes, spheres, cubes)
* **Meshes**
* **URDF** files (Universal Robotics Description Format)
* **MJCF** files (MuJoCo's Robotics Format)
* **Terrains**
* **Soft robot descriptions**

This flexibility means that most commonly used robotics models—such as those described in URDF or MJCF—can be seamlessly integrated into Genesis.

In [4]:
# Load Entity

plane = scene.add_entity(
    gs.morphs.Plane(),
)
franka = scene.add_entity(
    gs.morphs.MJCF(file="xml/franka_emika_panda/panda.xml"),
)

[Genesis] [09:55:54] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <cfe02cd>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [09:55:55] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <7a1c91d>, morph: <gs.morphs.MJCF(file='/opt/conda/envs/py_3.12/lib/python3.12/site-packages/genesis/assets/xml/franka_emika_panda/panda.xml')>, material: <gs.materials.Rigid>.
[Genesis] [09:55:55] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint1`
[Genesis] [09:55:55] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias for joint `finger_joint1`.
[Genesis] [09:55:55] [WARNING] (MJCF) Approximating tendon by joint actuator for `finger_joint2`
[Genesis] [09:55:55] [WARNING] (MJCF) Actuator control gain and bias parameters cannot be reduced to a unique PD control position gain. Using max between gain and bias for joint `finger_joint2`.
[Genesis] [09:55:55] [INFO] Applying o

## Add a Camera

Next, we add a camera to record what happens in the scene.

In [5]:
cam = scene.add_camera(
    res=(640, 480),
    pos=(3.5, 0.0, 2.5),
    lookat=(0, 0, 0.5),
    fov=30,
    GUI=True,
)

print("Successfully loaded a camera.")

Successfully loaded a camera.


## Build the Scene

After creating a scene, it must be built explicitly by calling `scene.build()`. This step is required because Genesis uses just-in-time (JIT) compilation to generate GPU kernels on the fly. Building the scene initializes this process, allocates device memory, and sets up the underlying data structures required for simulation.

In [6]:
scene.build()

print("Successfully built the scene.")

[Genesis] [09:56:05] [INFO] Building scene <17b8005>...
[Genesis] [09:56:06] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [09:56:06] [WARNING] Constraint solver time constant should be greater than 2*substep_dt. timeconst is changed from `0.005` to `0.02`). Decrease simulation timestep or increase timeconst to avoid altering the original value.
[Genesis] [09:56:07] [INFO] Compiling simulation kernels...
[Genesis] [09:56:10] [INFO] Building visualizer...
Successfully built the scene.


amdgpu: os_same_file_description couldn't determine if two DRM fds reference the same file description.
If they do, bad things may happen!


## Start simulating

Then we start simulating and save it as a video. 
You can download the result in `Video` folder.

In [7]:
# render rgb, depth, segmentation, normal
rgb, depth, segmentation, normal = cam.render(rgb=True, depth=True, segmentation=True, normal=True)
cam.start_recording()

for _ in range(100):
    scene.step()
    cam.render()
    
cam.stop_recording(save_to_filename="Videos/video_01.mp4", fps=60)

[Genesis] [09:56:11] [INFO] Running at 38.68 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.71 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.57 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.50 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.60 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.71 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.75 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.83 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.89 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.95 FPS.
[Genesis] [09:56:11] [INFO] Running at 38.98 FPS.
[Genesis] [09:56:11] [INFO] Running at 39.10 FPS.
[Genesis] [09:56:11] [INFO] Running at 39.17 FPS.
[Genesis] [09:56:12] [INFO] Running at 39.05 FPS.
[Genesis] [09:56:12] [INFO] Running at 39.12 FPS.
[Genesis] [09:56:12] [INFO] Running at 39.20 FPS.
[Genesis] [09:56:12] [INFO] Running at 39.22 FPS.
[Genesis] [09:56:12] [INFO] Running at 39.13 FPS.
[Genesis] [09:56:12] [INFO] Running at 39.24 FPS.
[Genesis] [09:56:12] [INFO] Running at 39.38 FPS.


## Show the video

If everything works correctly, you will see a robotic arm appear on the screen and naturally fall due to gravity.

In [8]:
from IPython.display import Video
Video(url="Videos/video_01.mp4")